In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/coot-videotext-master/NEW

/content/drive/MyDrive/coot-videotext-master/NEW


In [ ]:
ls

config/            meta_all.json   Model_training.ipynb  provided_models/
data/              metaSUMMe.json  nntrainer/            summaries.h5
experiments/       metaTVsum.json  pre-processing.ipynb
frms_per_seg.json  models/         pretrained_models/


In [ ]:
import json
import torch.nn as nn
import numpy as np
import h5py
import math
from tqdm import tqdm


In [ ]:
! pip install GPUtil
! pip install pathspec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from nntrainer.models.transformer_legacy import TransformerLegacy
from nntrainer.models.transformer_legacy import TransformerConfig

In [ ]:
transf_config = {'name': 'transformer', 'output_dim': 384, 'use_input_fc': True, 'input_fc_config': {'output_dim': 384, 'num_layers': 1, 'hidden_dim': 0, 'activation_middle': 'none', 'activation_output': 'gelu', 'dropout_middle': 0, 'dropout_output': 0, 'norm_middle': 'none', 'norm_output': 'none', 'residual': 'none'}, 'positional_encoding': 'sincos', 'add_local_cls_token': False, 'dropout_input': 0, 'norm_input': 'layernorm_coot', 'selfatn_config': {'hidden_dim': 384, 'num_layers': 1, 'num_heads': 8, 'pointwise_ff_dim': 384, 'activation': 'gelu', 'dropout': 0.05, 'norm': 'layernorm_coot'}, 'use_context': False, 'use_output_fc': False, 'pooler_config': {'name': 'atn', 'hidden_dim': 768, 'num_heads': 2, 'num_layers': 1, 'dropout': 0.05, 'activation': 'gelu'}, 'weight_init_type': 'truncnorm', 'weight_init_std': 0.01}
current_cfg = TransformerConfig(transf_config)
net_video_local = TransformerLegacy(current_cfg, 512)

In [ ]:
net_video_local.load_state_dict(torch.load('provided_models/Net_local.pth'))
net_video_local.eval()
net_video_local.to('cuda')

TransformerLegacy(
  (norm_input): LayerNormalization()
  (input_fc): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=512, out_features=384, bias=True)
    )
    (activation_output): GELU(approximate='none')
    (norm_output): Identity()
  )
  (embedding): PositionalEncodingSinCos(
    (dropout): Dropout(p=0, inplace=False)
  )
  (tf): TransformerEncoder(
    (encoder_layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attention_layer): Sublayer(
          (sublayer): MultiHeadAttention(
            (query_projection): Linear(in_features=384, out_features=384, bias=True)
            (key_projection): Linear(in_features=384, out_features=384, bias=True)
            (value_projection): Linear(in_features=384, out_features=384, bias=True)
            (final_projection): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.05, inplace=False)
            (softmax): Softmax(dim=3)
          )
          (layer_normalization): Lay

In [ ]:
transf_config = {'name': 'transformer', 'output_dim': 768, 'use_input_fc': False, 'input_fc_config': None, 'use_context': True, 'crossatn_config': {'hidden_dim': 384, 'num_layers': 1, 'num_heads': 8, 'pointwise_ff_dim': 384, 'activation': 'gelu', 'dropout': 0.05, 'norm': 'layernorm_coot'}, 'pooler_config': {'name': 'avg_special'}, 'positional_encoding': 'sincos', 'add_local_cls_token': False, 'dropout_input': 0, 'norm_input': 'layernorm_coot', 'selfatn_config': {'hidden_dim': 384, 'num_layers': 1, 'num_heads': 8, 'pointwise_ff_dim': 384, 'activation': 'gelu', 'dropout': 0.05, 'norm': 'layernorm_coot'}, 'use_output_fc': False, 'weight_init_type': 'truncnorm', 'weight_init_std': 0.01}
current_cfg = TransformerConfig(transf_config)
coot = TransformerLegacy(current_cfg, 384)

In [ ]:
coot.load_state_dict(torch.load('provided_models/TheModel.pth'))
coot.eval()
coot.to('cuda')

TransformerLegacy(
  (norm_input): LayerNormalization()
  (embedding): PositionalEncodingSinCos(
    (dropout): Dropout(p=0, inplace=False)
  )
  (tf): TransformerEncoder(
    (encoder_layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attention_layer): Sublayer(
          (sublayer): MultiHeadAttention(
            (query_projection): Linear(in_features=384, out_features=384, bias=True)
            (key_projection): Linear(in_features=384, out_features=384, bias=True)
            (value_projection): Linear(in_features=384, out_features=384, bias=True)
            (final_projection): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.05, inplace=False)
            (softmax): Softmax(dim=3)
          )
          (layer_normalization): LayerNormalization()
        )
        (pointwise_feedforward_layer): Sublayer(
          (sublayer): PointwiseFeedForwardNetwork(
            (feed_forward): Sequential(
              (0): Linear(in

## Summarizer model

In [ ]:
def knapsack_dp(values,weights,n_items,capacity,device='cpu'):
    # check_inputs(values,weights,n_items,capacity)
    
    # assert(isinstance(values,list))
    # assert(isinstance(weights,list))
    # assert(isinstance(n_items,int))
    # assert(isinstance(capacity,int))
    # check value type
    # assert(all(isinstance(val,int) or isinstance(val,float) for val in values))
    # assert(all(isinstance(val,int) for val in weights))
    # check validity of value
    # assert(all(val >= 0 for val in weights))
    # assert(n_items > 0)
    # assert(capacity > 0)

    table = torch.zeros((n_items+1,capacity+1)).to(device)
    keep = torch.zeros((n_items+1,capacity+1)).to(device)

    for i in range(1,n_items+1):
        for w in range(0,capacity+1):
            wi = weights[i-1] # weight of current item
            vi = values[i-1] # value of current item
            if (wi <= w) and (vi + table[i-1,w-wi] > table[i-1,w]):
                table[i,w] = vi*1 + table[i-1,w-wi]
                keep[i,w] = 1
            else:
                table[i,w] = vi*0+table[i-1,w]
    
    
    picks = []
    cum_wghts = torch.zeros(n_items).to(device)
    K = capacity
    for i in range(n_items,0,-1):
        if keep[i,K] == 1:
            picks.append(i)
            cum_wghts[i-1] = table[i,K]
            K -= weights[i-1]

    picks.sort()
    picks = [x-1 for x in picks] # change to 0-index
    chosens = (cum_wghts-torch.min(cum_wghts)) / (torch.max(cum_wghts)-torch.min(cum_wghts))
    # print(picks, torch.ceil(chosens) )
    

    # if return_all:
    #     max_val = table[n_items,capacity]
    #     return picks,max_val
    return picks, torch.ceil(chosens)

In [ ]:
class ShotScore(nn.Module):
    def __init__(self, input_size=384):
        super(ShotScore, self).__init__()
        self.Clip_Encoding = nn.Sequential(
                                nn.TransformerEncoderLayer(d_model=input_size, nhead=8, batch_first=True, activation = 'gelu'),
                                nn.TransformerEncoderLayer(d_model=input_size, nhead=8, batch_first=True, activation = 'gelu'))
        self.Clip_Scoring = nn.Sequential(
                                nn.Linear(in_features=input_size, out_features=input_size//2, bias=True),
                                nn.Linear(in_features=input_size//2, out_features=1, bias=True))
        
        self.softmax = nn.Softmax(dim=1)
        
    
    
    def forward(self,clip_embs, stages,  batch_mask=None):
        
        f1 = clip_embs
#         for s in range(stages):
#             clip_encoder = self.Clip_Encoding(Features)+Features
#             clip_Scores = self.Clip_Scoring(clip_encoder)

#     #         Scene_emb_list = []
#     #         for start, end in scene_boundaries:
#     #             scene_embs = self.scene_embs(clip_embs[start:end])
#     #             Scene_emb_list.append(scene_embs)

#     #             Scene_Scores = self.Scene_Scoring_Transformer(Scene_emb_list)

#     #         for start, end in scene_boundaries:  
#                 # clip_Scores+=Scene_Scores

#             si = self.softmax(clip_Scores)
#             progressive_scores.append(si)
#             Features += si*Features
        
        s1 = self.softmax(self.Clip_Scoring(self.Clip_Encoding(f1)+f1))
        
        f2 = s1*f1 + f1
        s2 = self.softmax(self.Clip_Scoring(self.Clip_Encoding(f2)+f2))
        
        f3 = s2*f2 + f2
        s3 = self.softmax(self.Clip_Scoring(self.Clip_Encoding(f3)+f3))
            
        
        scores = self.softmax(s1)
        return scores

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(features))
        self.beta = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

class SelfAttention(nn.Module):

    def __init__(self, apperture=-1, ignore_itself=False, input_size=1024, output_size=1024):
        super(SelfAttention, self).__init__()

        self.apperture = apperture
        self.ignore_itself = ignore_itself

        self.m = input_size
        self.output_size = output_size

        self.K = nn.Linear(in_features=self.m, out_features=self.output_size, bias=False)
        self.Q = nn.Linear(in_features=self.m, out_features=self.output_size, bias=False)
        self.V = nn.Linear(in_features=self.m, out_features=self.output_size, bias=False)
        self.output_linear = nn.Linear(in_features=self.output_size, out_features=self.m, bias=False)

        self.drop50 = nn.Dropout(0.5)

    def forward(self, x):
        n = x.shape[0]  # sequence length

        K = self.K(x)  # ENC (n x m) => (n x H) H= hidden size
        Q = self.Q(x)  # ENC (n x m) => (n x H) H= hidden size
        V = self.V(x)

        Q *= 0.06
        logits = torch.matmul(Q, K.transpose(1,0))

        if self.ignore_itself:
            # Zero the diagonal activations (a distance of each frame with itself)
            logits[torch.eye(n).byte()] = -float("Inf")

        if self.apperture > 0:
            # Set attention to zero to frames further than +/- apperture from the current one
            onesmask = torch.ones(n, n)
            trimask = torch.tril(onesmask, -self.apperture) + torch.triu(onesmask, self.apperture)
            logits[trimask == 1] = -float("Inf")

        att_weights_ = nn.functional.softmax(logits, dim=-1)
        weights = self.drop50(att_weights_)
        y = torch.matmul(V.transpose(1,0), weights).transpose(1,0)
        y = self.output_linear(y)

        return y, att_weights_


class VASNet(nn.Module):

    def __init__(self):
        super(VASNet, self).__init__()

        self.m = 384 # cnn features size
        self.hidden_size = 384

        self.att = SelfAttention(input_size=self.m, output_size=self.m)
        self.ka = nn.Linear(in_features=self.m, out_features=384)
        self.kb = nn.Linear(in_features=self.ka.out_features, out_features=384)
        self.kc = nn.Linear(in_features=self.kb.out_features, out_features=384)
        self.kd = nn.Linear(in_features=self.ka.out_features, out_features=1)

        self.sig = nn.Sigmoid()
        self.relu = nn.GELU()
        self.drop50 = nn.Dropout(0.5)
        self.softmax = nn.Softmax(dim=0)
        self.layer_norm_y = LayerNorm(self.m)
        self.layer_norm_ka = LayerNorm(self.ka.out_features)


    def forward(self, x):

        m = x.shape[2] # Feature size

        # Place the video frames to the batch dimension to allow for batch arithm. operations.
        # Assumes input batch size = 1.
        x = x.view(-1, m)
        y, att_weights_ = self.att(x)

        y = y + x
        y = self.drop50(y)
        y = self.layer_norm_y(y)

        # Frame level importance score regression
        # Two layer NN
        y = self.ka(y)
        y = self.relu(y)
        y = self.drop50(y)
        y = self.layer_norm_ka(y)

        # y = self.kd(y)
        # y = self.sig(y)
        # y = y.view(1, -1)

        return y


class ShotScore2(nn.Module):
    def __init__(self, input_size=384):
        super(ShotScore2, self).__init__()
        self.Clip_Encoding = nn.Sequential(
                                nn.TransformerEncoderLayer(d_model=input_size, nhead=8, batch_first=True, activation = 'gelu'),
                                nn.TransformerEncoderLayer(d_model=input_size, nhead=8, batch_first=True, activation = 'gelu'))
        self.Clip_Scoring = nn.Sequential(
                                nn.Linear(in_features=input_size, out_features=input_size//2, bias=True),
                                nn.Linear(in_features=input_size//2, out_features=1, bias=True))
        
        self.softmax = nn.Softmax(dim=1)

        self.vasnet = VASNet()
        
    
    
    def forward(self,clip_embs, stages,  batch_mask=None):
        
        f1 = clip_embs
#         for s in range(stages):
#             clip_encoder = self.Clip_Encoding(Features)+Features
#             clip_Scores = self.Clip_Scoring(clip_encoder)

#     #         Scene_emb_list = []
#     #         for start, end in scene_boundaries:
#     #             scene_embs = self.scene_embs(clip_embs[start:end])
#     #             Scene_emb_list.append(scene_embs)

#     #             Scene_Scores = self.Scene_Scoring_Transformer(Scene_emb_list)

#     #         for start, end in scene_boundaries:  
#                 # clip_Scores+=Scene_Scores

#             si = self.softmax(clip_Scores)
#             progressive_scores.append(si)
#             Features += si*Features
        
        s1 = self.softmax(self.Clip_Scoring(self.vasnet(f1)+f1))
        
        
        # f2 = s1*f1 + f1
        # s2 = self.softmax(self.Clip_Scoring(self.vasnet(f2)+f2))
        
        
        # f3 = s2*f2 + f2
        # s3 = self.softmax(self.Clip_Scoring(self.vasnet(f3)+f3))
            
        scores = self.softmax(s1)
        return scores

## Trainer

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname == 'Linear':
        torch.nn.init.xavier_uniform_(m.weight, gain=np.sqrt(2.0))
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0.1)

In [ ]:
def get_hidden_context(key, pred_clip_mask, nfps, device='cpu'):
    with h5py.File('data/youcook2/video_feat_100m.h5', 'r') as d:
        vid_feats = torch.tensor(d[key][()]).to(device)
        mask = torch.zeros(len(vid_feats)).to(device)
        
        # start = 0
        # for i in range(len(nfps)):
        #     end = nfps[i]+start
        #     if i in picks:
        #         mask[start:end]=1
        #     start = end
        
        start = 0
        for i in range(len(nfps)):
            end = nfps[i]+start
            mask[start:end]=pred_clip_mask[i]
            start = end
            

        masked_vidfeats = vid_feats*mask.unsqueeze(0).T
        masked_vidfeats = masked_vidfeats.unsqueeze(0)

        # print(masked_vidfeats, vid_feats, mask)
        # masked_vidfeats = masked_vidfeats[masked_vidfeats.sum(dim=2) != 0].unsqueeze(0)
        # nfs = mask.sum().int()
        nfs = len(vid_feats)
        mask = torch.zeros(1, nfs).bool().to(device)
        vid_context,_ = net_video_local(masked_vidfeats, mask, torch.tensor(nfs).int(), None)
        return vid_context
    
    

In [ ]:
cos_sim = torch.nn.CosineSimilarity(dim=0, eps=0)
def diversity_loss(masked_vid):
    agg = []
    dl = 0
    for i in range(masked_vid.shape[0]):
        for j in range(i+1, masked_vid.shape[0]):
            yi = masked_vid[i]
            yj = masked_vid[j]
            cs = cos_sim(yi, yj)
            agg.append(cs)
            dl+=cs
            
    return torch.mean(torch.stack(agg))
        



In [ ]:
Summe = ['Base jumping', 'Bearpark_climbing', 'Bearpark_climbing', 'Bus_in_Rock_Tunnel', 'car_over_camera', 'Car_railcrossing', 'Cockpit_Landing',  'Cooking', 'Eiffel Tower', 'Excavators river crossing', 'Fire Domino', 'Jumps', 'Kids_playing_in_leaves','Notre_Dame', 'Paintball', 'paluma_jump', 'playing_ball', 'Playing_on_water_slide', 'Saving dolphines', 'Scuba', 'St Maarten Landing', 'Statue of Liberty', 'Uncut_Evening_Flight','Valparaiso_Downhill', 'Air_Force_One']
len(Summe)

25

In [ ]:
def train(model, batch_size=1, device='cpu'):
    with open('frms_per_seg.json', 'r') as file:
        NFPS = json.load(file)
    l = nn.MSELoss().to(device)
    elu = nn.ELU()
    ScoreModel = model
    parameters = filter(lambda p: p.requires_grad, ScoreModel.parameters())
    optimizer = torch.optim.AdamW(parameters)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2)
    prev_mask=0
    batch_loss = []
    with h5py.File('experiments/retrieval/paper2020/yc2_100m_coot_valset1/embeddings/embeddings_0.h5', 'r') as d:
        clips = d['clip_num'][...]
        
        for epoch in range(50):
            pointer = 0
            epoch_loss = []
            
            print('Epoch:', epoch,'/50')
            pbar = tqdm(total=len(clips), position=0, leave=True)
            for clip_num, nclips in enumerate(clips):
                key = d['key'][clip_num].decode()
                
                clip_embs = d['clip_emb'][pointer:pointer+nclips]
                ori_vid_cont = d['vid_context'][clip_num]
                ori_vid_emb = d['vid_emb'][clip_num]
                pointer += nclips
                
                if key in Summe:
                  continue

                # vid_cont = torch.tensor(vid_context).unsqueeze(0)
                clip_embs = torch.tensor(clip_embs).unsqueeze(0).to(device)
                ori_vid_cont = torch.tensor(ori_vid_cont).unsqueeze(0).to(device)
                ori_vid_emb = torch.tensor(ori_vid_emb).unsqueeze(0).to(device)


                scores = ScoreModel(clip_embs,3).reshape(-1)

                nfps = NFPS[key]

                picks, predicted_mask = knapsack_dp(scores.squeeze(0), NFPS[key], len(nfps), math.ceil(sum(nfps)*15/100), device=device)

                


                # print('###')
                # predicted_mask = torch.zeros(nclips)
                # predicted_mask[picks]=1

                # mask = torch.ceil(predicted_mask.unsqueeze(0)*scores)
                maskr = predicted_mask.unsqueeze(0)
                maskc = predicted_mask-1

                s1 = torch.ceil(scores*maskr)

                s2 = elu(scores*maskc)
                mask = torch.abs(s1+s2)

                vid_cont = get_hidden_context(key, mask.squeeze(), nfps, device)


                # if clip_num==0:
                #     print(mask-prev_mask)
                #     prev_mask=mask
                    
                masked_vid = clip_embs*mask.T

                # masked_vid = masked_vid[masked_vid.sum(dim=2) != 0].unsqueeze(0)

                ### Check masked_vid has grads
                # clipn = predicted_mask.sum().int()
                clipn = nclips
                cootmask = torch.zeros(1, clipn).bool().to(device)
                
                vid_emb, _ = coot(masked_vid, cootmask, torch.tensor(clipn).int().to(device), vid_cont)
                # print(ori_vid_emb.shape)
                # print(vid_emb.shape)

                selected_clips = masked_vid*predicted_mask.unsqueeze(0).T
                selected_clips = selected_clips[selected_clips.sum(dim=2) != 0].unsqueeze(0)

                div_loss = diversity_loss(selected_clips.squeeze())
                reconstruction_loss = l(vid_emb, ori_vid_emb)
                context_loss  = l(vid_cont, ori_vid_cont)

                Loss = (0.1*div_loss) + (0.9*context_loss)

                batch_loss.append(Loss.unsqueeze(0)) 

                if(len(batch_loss) == 5):
                  BLoss = torch.mean(torch.cat(batch_loss))
                  optimizer.zero_grad()
                  BLoss.backward()
                  optimizer.step()
                  scheduler.step(BLoss)
                  batch_loss = []

            #     optimizer.zero_grad()
            #     Loss.backward()
            #     optimizer.step()

                epoch_loss.append(Loss.item())
                
                pbar.update(1)
                pbar.set_description("Loss= %.3f" % Loss.item())
            pbar.close()
                
            print('Average Epoch Loss:', sum(epoch_loss)/len(epoch_loss))
            torch.save(ScoreModel.state_dict(), f'models/model@{epoch%5}.pth')
            
            if(epoch%1==0):
                fs_gt, fs_us = validation(ScoreModel, device='cuda')
                print('Avg GT FSore:',fs_gt, '   Avg User FSore:', fs_us)
            
            
    return(ScoreModel)
    
            

## Validation

In [ ]:
def validation(ScoreModel, device='cpu'):
    
    with open('frms_per_seg.json', 'r') as file:
        NFPS = json.load(file)
        
    avg_gtfs = []
    avg_usfs = []
    with h5py.File('experiments/retrieval/paper2020/yc2_100m_coot_valset1/embeddings/embeddings_0.h5', 'r') as d, h5py.File('summaries.h5','r') as summh5:
        clips = d['clip_num'][...]

        pointer = 0
        for clip_num, nclips in enumerate(clips):
            key = d['key'][clip_num].decode()
            if key in Summe:
              continue
            clip_embs = d['clip_emb'][pointer:pointer+nclips]
            pointer += nclips

            clip_embs = torch.tensor(clip_embs).unsqueeze(0).to(device)

            scores = ScoreModel(clip_embs, 3).reshape(-1).cpu()

            nfps = NFPS[key]

            picks,_ = knapsack_dp(scores.squeeze(0), NFPS[key], len(nfps), math.ceil(sum(nfps)*15/100))
            
            machine_summary = get_machine_summ(picks, nfps)
            gt_summary = summh5[key]['gt_summary']
            user_summary = summh5[key]['user_summary']
            fs_gt,_,_ = evaluate_summary(machine_summary, np.expand_dims(gt_summary, 0) )
            fs_ut,_,_ = evaluate_summary(machine_summary, np.array(user_summary) )
            avg_gtfs.append(fs_gt)
            avg_usfs.append(avg_usfs)
            
        return fs_gt.mean(), fs_ut.mean()


In [ ]:
def get_machine_summ(picks, nfps):
    tot_feats = sum(nfps)
    summ = []
    for n, nfs in enumerate(nfps):
        if n in picks:
            summ = np.append(summ, np.ones(16*nfs))
        else:
            summ = np.append(summ, np.zeros(16*nfs))
    return summ

In [ ]:
def evaluate_summary(machine_summary, user_summary, eval_metric='avg'):
    """Compare machine summary with user summary (keyshot-based).
    Args:
    --------------------------------
    machine_summary and user_summary should be binary vectors of ndarray type.
    eval_metric = {'avg', 'max'}
    'avg' averages results of comparing multiple human summaries.
    'max' takes the maximum (best) out of multiple comparisons.
    """
    machine_summary = machine_summary.astype(np.float32)
    user_summary = user_summary.astype(np.float32)
    n_users,n_frames = user_summary.shape

    # binarization
    machine_summary[machine_summary > 0] = 1
    user_summary[user_summary > 0] = 1

    if len(machine_summary) > n_frames:
        machine_summary = machine_summary[:n_frames]
    elif len(machine_summary) < n_frames:
        zero_padding = np.zeros((n_frames - len(machine_summary)))
        machine_summary = np.concatenate([machine_summary, zero_padding])

    f_scores = []
    prec_arr = []
    rec_arr = []

    for user_idx in range(n_users):
        gt_summary = user_summary[user_idx,:]
        overlap_duration = (machine_summary * gt_summary).sum()
        precision = overlap_duration / (machine_summary.sum() + 1e-8)
        recall = overlap_duration / (gt_summary.sum() + 1e-8)
        if precision == 0 and recall == 0:
            f_score = 0.
        else:
            f_score = (2 * precision * recall) / (precision + recall)
        f_scores.append(f_score)
        prec_arr.append(precision)
        rec_arr.append(recall)

    final_f_score = np.mean(f_scores)
    final_prec = np.mean(prec_arr)
    final_rec = np.mean(rec_arr)
    return final_f_score, final_prec, final_rec

## Testing

In [ ]:
model = ShotScore2()
# model.load_state_dict(torch.load('models/model@1.pth', map_location=lambda storage, loc: storage)) 
model.apply(weights_init)
model.to('cuda')
trained_model = train(model, device='cuda')

Epoch: 0 /50


Loss= 0.600:  67%|██████▋   | 48/72 [01:09<00:34,  1.46s/it]


Average Epoch Loss: 0.5745956897735596
Avg GT FSore: 0.3489148580910031    Avg User FSore: 0.2851319676361825
Epoch: 1 /50


Loss= 0.598:  67%|██████▋   | 48/72 [01:16<00:38,  1.59s/it]


Average Epoch Loss: 0.574719017992417
Avg GT FSore: 0.44090528079730246    Avg User FSore: 0.2852468524256314
Epoch: 2 /50


...

Average Epoch Loss: 0.5749880547324816
Avg GT FSore: 0.43739565942508524    Avg User FSore: 0.3376620296134274
Epoch: 8 /50


Loss= 0.598:  67%|██████▋   | 48/72 [01:11<00:35,  1.50s/it]


Average Epoch Loss: 0.5748485041161379
Avg GT FSore: 0.28714524206532316    Avg User FSore: 0.23829048693498586
Epoch: 9 /50


Loss= 0.564:  54%|█████▍    | 39/72 [00:54<01:08,  2.07s/it]

KeyboardInterrupt: ignored

In [ ]:
                                                                                    Sairam Sairam Sairam